# Velocity Based Training Notebook

Notebook which gives you the rep velocity from a given input video (pause, ascent and descent) also available, make sure you have detectron2 installed


In [ ]:
!wget http://www.purestrength.ai/opensource/final_model_3class.pth

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

create random folder to allow the video to be converted into images to run analysis onto

In [1]:
import urllib
import os
import cv2
import shutil
from time import time
import numpy as np
rng = np.random.default_rng()
val = rng.integers(low=0,high=1000000,size=1)[0]
temp_dir = str(val)

In [2]:
# temp_dir = 'temporary'
try:
    os.mkdir(temp_dir)
except:
    pass


In [3]:
filename = 'diego205.mov'

In [4]:
shutil.copyfile(filename, os.path.join(temp_dir,filename))

'74334/diego205.mov'

In [5]:
#filename = split_name[1].split('%40')[0]+'_'+split_name[3].split('_')[0]+'_at'+split_name[2].split('_')[0]+'_'+split_name[4]

# split_name = clean_links.split('%3D')
files = os.listdir(temp_dir)
new_list = []
clean_folders = []
from time import time
for file in files:
    if any(s in file.lower() for s in ['mp4','avi','mov']):
        new_list.append(file)
    else:
        print('{} not an allowed filetype'.format(file))
files = new_list
print(files)
t0 = time()
for f in files:
    print('running on file {}'.format(f))
    if 'mp4' in f:
        folder_name = f.split('.mp4')[0]
    elif 'mov' in f:
        folder_name = f.split('.mov')[0]
    elif 'avi' in f:
        folder_name = f.split('.avi')[0]
    else:
        print('warning which filetype is this??')
        folder_name = f.split('.')[0]
    folder_name = folder_name.replace(' ','-')
    folder_name = folder_name.replace('_', '-')
    clean_folders.append(folder_name+'_')
    print(folder_name+'_')
    print('outputs/{}/{}.mov'.format(temp_dir, folder_name+'_.mov'))
    try:
        os.mkdir(temp_dir+'/{}_'.format(folder_name))
        print(temp_dir+'/{}_'.format(f))
        vidcap = cv2.VideoCapture(temp_dir+'/{}'.format(f))
        success, image = vidcap.read()
        count = 0
        while success:
            cv2.imwrite(temp_dir+"/{}_/%04d.jpg".format(folder_name) % count, image)  # save frame as JPEG file
            success, image = vidcap.read()
            print('Read a new frame: ', success)
            count += 1
    except:
            print('folders already created moving to the next step')
print(time()-t0)

['diego205.mov']
running on file diego205.mov
diego205_
outputs/74334/diego205_.mov.mov
74334/diego205.mov_
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read

In [32]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer
from detectron2.config import get_cfg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.figure import Figure
from mpl_toolkits.axisartist.axislines import Subplot 


#if we just want to load a saved model
cfg = get_cfg()
cfg.merge_from_file("mask_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.55 # Set threshold for this model
cfg.MODEL.WEIGHTS = 'final_model_3class.pth' # Set path model .pth
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
predictor = DefaultPredictor(cfg)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


### Barbell Analysis

In [7]:
import os
from time import time

folder = temp_dir+'/'+filename[:len(filename)-4]+'_'

files = os.listdir(folder)
files = sorted(files, key = lambda x:int(x.split('.')[0]))

centroid_topleft_info = {}
for i in range(0,24):
    centroid_topleft_info[i] = []
bar_info = {}
bar_info_predmask = {}

    
import matplotlib.patches as patches
def bbox2(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    return xmin,ymin,xmax-xmin,ymax-ymin
import cv2
zeros = '0000'
t0 = time()
for file in files:
    print(file)
    idx= int(file.split('.')[0])
    im = cv2.imread('{}/{}'.format(folder,file))
    outputs = predictor(im)
    predictions = outputs["instances"].to("cpu")
    """bar_info structure -> x_start,y_start,x_end,y_end"""
    bar_info[idx] = {}
    bar_info_predmask[idx] = {}
    for i in range(0,len(predictions)):
        if i == 0:
            bar = predictions[i].pred_masks[0,:,:]
        else:
            bar += predictions[i].pred_masks[0,:,:]
        bar_info[idx][i] = predictions[i].pred_boxes
        xmin,ymin,w,h = bbox2(predictions[i].pred_masks.numpy()[0])
        try:
            bar_info_predmask[idx][predictions[i].pred_classes.numpy()[0]].append([xmin,ymin,w,h])
        except:
            bar_info_predmask[idx][predictions[i].pred_classes.numpy()[0]] = [[xmin,ymin,w,h]]
        try:
            bar_info_predmask[idx][predictions[i].pred_classes.numpy()[0]].sort(key=lambda x:x[2]*x[3], reverse=True)
        except:
            pass

print(time()-t0)




0000.jpg


/Users/diegogranziol/opt/anaconda3/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


0001.jpg
0002.jpg
0003.jpg
0004.jpg
0005.jpg
0006.jpg
0007.jpg
0008.jpg
0009.jpg
0010.jpg
0011.jpg
0012.jpg
0013.jpg
0014.jpg
0015.jpg
0016.jpg
0017.jpg
0018.jpg
0019.jpg
0020.jpg
0021.jpg
0022.jpg
0023.jpg
0024.jpg
0025.jpg
0026.jpg
0027.jpg
0028.jpg
0029.jpg
0030.jpg
0031.jpg
0032.jpg
0033.jpg
0034.jpg
0035.jpg
0036.jpg
0037.jpg
0038.jpg
0039.jpg
0040.jpg
0041.jpg
0042.jpg
0043.jpg
0044.jpg
0045.jpg
0046.jpg
0047.jpg
0048.jpg
0049.jpg
0050.jpg
0051.jpg
0052.jpg
0053.jpg
0054.jpg
0055.jpg
0056.jpg
0057.jpg
0058.jpg
0059.jpg
0060.jpg
0061.jpg
0062.jpg
0063.jpg
0064.jpg
0065.jpg
0066.jpg
0067.jpg
0068.jpg
0069.jpg
0070.jpg
0071.jpg
0072.jpg
0073.jpg
0074.jpg
0075.jpg
0076.jpg
0077.jpg
0078.jpg
0079.jpg
0080.jpg
0081.jpg
0082.jpg
0083.jpg
0084.jpg
0085.jpg
0086.jpg
0087.jpg
0088.jpg
0089.jpg
0090.jpg
0091.jpg
0092.jpg
0093.jpg
0094.jpg
0095.jpg
0096.jpg
0097.jpg
0098.jpg
0099.jpg
0100.jpg
0101.jpg
0102.jpg
0103.jpg
0104.jpg
0105.jpg
0106.jpg
0107.jpg
0108.jpg
0109.jpg
0110.jpg
0111.jpg
0

In [33]:
outputs = predictor(im)

In [34]:
outputs

{'instances': Instances(num_instances=0, image_height=1244, image_width=916, fields=[pred_boxes: Boxes(tensor([], size=(0, 4))), scores: tensor([]), pred_classes: tensor([], dtype=torch.int64), pred_masks: tensor([], size=(0, 1244, 916), dtype=torch.bool)])}

In [20]:
# missing_list = 0
# thing = []
# idx_choice = 2
# for a in range(0,len(bar_info_predmask)):
#     try:
#         thing.append([bar_info_predmask[a][idx_choice][0][0]+(bar_info_predmask[a][idx_choice][0][2]//2),-bar_info_predmask[a][idx_choice][0][1]-(bar_info_predmask[a][idx_choice][0][3]//2)])
#     except:
#         print(a)
#         missing_list += 1
# var = 0
# for i in range(0,len(thing[0])):
#     if np.var([a[i] for a in thing])>var:
#         var = np.var([a[i] for a in thing])
#         movement_direction=i
# print(movement_direction)
# print('total fraction of missing frames = {}'.format(missing_list/len(bar_info_predmask)))
# if missing_list/len(bar_info_predmask) > 0.01:
#     print('too many frames missing using weight center')
thing = []
missing_list = 0
thing = []
idx_choice = 0
for a in range(0,len(bar_info_predmask)):
    try:
        thing.append([bar_info_predmask[a][idx_choice][0][0]+(bar_info_predmask[a][idx_choice][0][2]//2),-bar_info_predmask[a][idx_choice][0][1]-(bar_info_predmask[a][idx_choice][0][3]//2)])
    except:
        print(a)
        missing_list += 1
var = 0
for i in range(0,len(thing[0])):
    if np.var([a[i] for a in thing])>var:
        var = np.var([a[i] for a in thing])
        movement_direction=i
print(movement_direction)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210


IndexError: list index out of range

In [22]:
bar_info_predmask

{0: {},
 1: {},
 2: {},
 3: {},
 4: {},
 5: {},
 6: {},
 7: {},
 8: {},
 9: {},
 10: {},
 11: {},
 12: {},
 13: {},
 14: {},
 15: {},
 16: {},
 17: {},
 18: {},
 19: {},
 20: {},
 21: {},
 22: {},
 23: {},
 24: {},
 25: {},
 26: {},
 27: {},
 28: {},
 29: {},
 30: {},
 31: {},
 32: {},
 33: {},
 34: {},
 35: {},
 36: {},
 37: {},
 38: {},
 39: {},
 40: {},
 41: {},
 42: {},
 43: {},
 44: {},
 45: {},
 46: {},
 47: {},
 48: {},
 49: {},
 50: {},
 51: {},
 52: {},
 53: {},
 54: {},
 55: {},
 56: {},
 57: {},
 58: {},
 59: {},
 60: {},
 61: {},
 62: {},
 63: {},
 64: {},
 65: {},
 66: {},
 67: {},
 68: {},
 69: {},
 70: {},
 71: {},
 72: {},
 73: {},
 74: {},
 75: {},
 76: {},
 77: {},
 78: {},
 79: {},
 80: {},
 81: {},
 82: {},
 83: {},
 84: {},
 85: {},
 86: {},
 87: {},
 88: {},
 89: {},
 90: {},
 91: {},
 92: {},
 93: {},
 94: {},
 95: {},
 96: {},
 97: {},
 98: {},
 99: {},
 100: {},
 101: {},
 102: {},
 103: {},
 104: {},
 105: {},
 106: {},
 107: {},
 108: {},
 109: {},
 110: {},


In [9]:
cap= cv2.VideoCapture(os.path.join(temp_dir,filename))

framespersecond = cap.get(cv2.CAP_PROP_FPS)

print(framespersecond)
# hand_motion_xy = thing
important_motion = [val[movement_direction] for val in thing]
# offset = idx-cutoff
start_val = important_motion[0]
ema_important_motion = [start_val]
smoothing_coef = 0.5
print('influence of frame dies after {} frames which is {} seconds'.format(1/(1-smoothing_coef),1/framespersecond*(1-smoothing_coef)))
smoothed_val = start_val
for i in range(0,len(important_motion)):
    smoothed_val = smoothing_coef*smoothed_val + (1-smoothing_coef)*important_motion[i]
    ema_important_motion.append(smoothed_val)
plt.plot(ema_important_motion)
plt.show()
plt.close()

30.04984571564206


IndexError: list index out of range

In [ ]:
tot = 0
x_vals = []
for idx in range(0,len(bar_info_predmask)):
    try:
        if len(bar_info_predmask[idx][0]) > tot:
            x_vals = []
            _x_vals = []
            for k in range(0,len(bar_info_predmask[idx][0])):
                _x_vals.append(bar_info_predmask[idx][0][k][0])
            x_vals.append(_x_vals)
            tot = max(len(bar_info_predmask[idx][0]),tot)
        elif len(bar_info_predmask[idx][0]) >= tot:
            _x_vals = []
            for k in range(0,len(bar_info_predmask[idx][0])):
                _x_vals.append(bar_info_predmask[idx][0][k][0])
            x_vals.append(_x_vals)
        else:
            pass
    except:
        pass
print(tot)

x_vals_new = []
for k in range(0,len(x_vals[0])):
    x_vals_new.append(np.mean([x[k] for x in x_vals]))
x_vals = x_vals_new

x_vals.sort(key=lambda x:x)

from collections import OrderedDict
d = OrderedDict()
heights = OrderedDict()
for vals in x_vals:
    d[vals] = []
    heights[vals] = []
missing_frames = 0
for idx in range(0,len(bar_info_predmask)):
    try:
        for k in range(0,len(bar_info_predmask[idx][0])):
            index = np.argmin([(x1-x2)**2 for x1,x2 in zip(x_vals,[bar_info_predmask[idx][0][k][0]]*len(x_vals))])
            d[x_vals[index]].append([bar_info_predmask[idx][0][k][0]+bar_info_predmask[idx][0][k][0]//2,-bar_info_predmask[idx][0][k][1]-bar_info_predmask[idx][0][k][3]//2])
            heights[x_vals[index]].append(bar_info_predmask[idx][0][k][3])
    except:
        missing_frames +=1
frac_frames_missing = missing_frames/len(bar_info_predmask)
from copy import deepcopy
initial_var = 0
for keys in d.keys():
    cap= cv2.VideoCapture(os.path.join(temp_dir,filename))
    framespersecond = cap.get(cv2.CAP_PROP_FPS)
    movement_direction = 1
    print(framespersecond)
    # hand_motion_xy = thing
    important_motion = [val[movement_direction] for val in d[keys]]
    # offset = idx-cutoff
    start_val = important_motion[0]
    ema_important_motion = [start_val]
    print('influence of frame dies after {} frames which is {} seconds'.format(1/(1-smoothing_coef),1/framespersecond*(1-smoothing_coef)))
    smoothed_val = start_val
    for i in range(0,len(important_motion)):
        smoothed_val = smoothing_coef*smoothed_val + (1-smoothing_coef)*important_motion[i]
        ema_important_motion.append(smoothed_val)
    if np.var(ema_important_motion) > initial_var:
        initial_var = np.var(ema_important_motion)
        final_ema = ema_important_motion
        height = np.mean(heights[keys])
print('total fraction of missing frames {:3g}'.format(frac_frames_missing))
# return final_ema,height,framespersecond, frac_frames_missing

In [ ]:
tolerance = 1
check = []
velocity = np.diff(ema_important_motion)
for i in range(0,len(velocity)):
    if abs(velocity[i]) < tolerance:
        check.append(i)
    
plt.plot(ema_important_motion)
x_marker = [point for point in check]
y_marker = [ema_important_motion[point] for point in check]
plt.plot(x_marker, y_marker, 'bo')
plt.show()

In [ ]:
y_diff = np.diff(y_marker)
x_marker_2 = []
tol = 0.5*np.max(abs(y_diff))
for i in range(0,len(y_diff)):
    if abs(y_diff[i])>tol:
        x_marker_2.append(x_marker[i])
        x_marker_2.append(x_marker[1+i])
plt.plot(ema_important_motion)
y_marker_2 = [ema_important_motion[point] for point in x_marker_2]
plt.plot(x_marker_2, y_marker_2, 'bo')
plt.savefig(os.path.join(temp_dir,'velocityinfo.png'), bbox_inches='tight')
plt.show()

bottom_reps = []
y_rom = []
miny,maxy = min(y_marker_2),max(y_marker_2)
rep_names = ['descent','pause hole','pause between reps','ascent']
rep_info = dict()
for rep in rep_names:
    rep_info[rep] = []
for i in range(1,len(x_marker_2)):
    if abs(maxy-y_marker_2[i-1])<abs(miny-y_marker_2[i-1]):
        # print('rep {} is on top'.format(i-1))
        if y_marker_2[i] < y_marker_2[i-1] and abs(maxy-y_marker_2[i])>abs(miny-y_marker_2[i]):
            # print('going down')
            rep_info['descent'].append((x_marker_2[i]-x_marker_2[i-1])/framespersecond)
        else:
            # print('resting between reps')
            rep_info['pause between reps'].append((x_marker_2[i]-x_marker_2[i-1])/framespersecond)
    else:
        # print('rep {} at bottom'.format(i-1))
        if y_marker_2[i] > y_marker_2[i-1] and abs(maxy-y_marker_2[i])<abs(miny-y_marker_2[i]):
            # print('rising')
            rep_info['ascent'].append((x_marker_2[i]-x_marker_2[i-1])/framespersecond)
            bottom_reps.append(x_marker_2[i-1])
            y_rom.append(y_marker_2[i]-y_marker_2[i-1])
        else:
            # print('pausing at the bottom')
            rep_info['pause hole'].append((x_marker_2[i]-x_marker_2[i-1])/framespersecond)
for info in rep_info.keys():
  print('{} {} \n'.format(info,rep_info[info]))

print('average ascent time = {} seconds'.format(np.mean(rep_info['ascent'])))